# Cross Validation

Este notebook implementa técnicas de validación cruzada para evaluar modelos de machine learning.

## Características:
- K-Fold Cross Validation
- Stratified K-Fold para problemas de clasificación
- Time Series Split para datos temporales
- Grid Search con Cross Validation
- Comparación de métricas entre diferentes folds
- Visualización de resultados de validación cruzada


In [ ]:
# Imports y configuración
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import (
    cross_val_score, KFold, StratifiedKFold, TimeSeriesSplit,
    GridSearchCV, cross_validate, validation_curve
)
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    mean_squared_error, r2_score, make_scorer
)
import warnings
warnings.filterwarnings('ignore')

# Configuración general de gráficos
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)


In [ ]:
# Configuración del dataset
DATA_PATH = Path(r"C:\Users\INMORTAL\OneDrive\Documentos\python\notebooks\modelos\Data\Global_Cybersecurity_Threats_2015-2024.csv")

# Cargar datos
df = pd.read_csv(DATA_PATH)
print("Shape del dataset:", df.shape)
print("Columnas:", list(df.columns))
df.head()


In [ ]:
# Preparación de datos para clasificación
print("=== PREPARACIÓN DE DATOS PARA CLASIFICACIÓN ===")

# Para clasificación: Attack Type como objetivo
y_class = df['Attack Type']
X_class = df.drop(columns=['Attack Type'])

# Mapear etiquetas no numéricas a códigos
if not pd.api.types.is_numeric_dtype(y_class):
    y_class = y_class.astype('category').cat.codes
    print(f"Mapeo de etiquetas a códigos aplicado sobre 'Attack Type'.")

# One-hot encoding para columnas categóricas en X
X_class = pd.get_dummies(X_class, drop_first=True)

print("Shape X_class:", X_class.shape, "- y_class:", y_class.shape)
print("Clases en y_class y conteo:")
print(y_class.value_counts())

# Escalado para clasificación
scaler_class = StandardScaler()
X_class_scaled = scaler_class.fit_transform(X_class)


In [ ]:
# Preparación de datos para regresión
print("\n=== PREPARACIÓN DE DATOS PARA REGRESIÓN ===")

# Para regresión: Financial Loss como objetivo
y_reg = df['Financial Loss (in Million $)']
X_reg = df[['Number of Affected Users', 'Year']]  # Variables numéricas simples

print("Shape X_reg:", X_reg.shape, "- y_reg:", y_reg.shape)
print("Estadísticas de y_reg:")
print(y_reg.describe())

# Escalado para regresión
scaler_reg = StandardScaler()
X_reg_scaled = scaler_reg.fit_transform(X_reg)


In [ ]:
# 1. K-Fold Cross Validation para Clasificación
print("=== K-FOLD CROSS VALIDATION PARA CLASIFICACIÓN ===")

# Crear el modelo
log_reg = LogisticRegression(max_iter=200, random_state=RANDOM_STATE)

# K-Fold Cross Validation
kfold = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

# Evaluar con diferentes métricas
scoring = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted']
cv_results = cross_validate(log_reg, X_class_scaled, y_class, 
                           cv=kfold, scoring=scoring, return_train_score=True)

print("Resultados K-Fold (5 folds):")
for metric in scoring:
    test_scores = cv_results[f'test_{metric}']
    train_scores = cv_results[f'train_{metric}']
    print(f"{metric}:")
    print(f"  Test:  {test_scores.mean():.4f} (+/- {test_scores.std() * 2:.4f})")
    print(f"  Train: {train_scores.mean():.4f} (+/- {train_scores.std() * 2:.4f})")
    print()


In [ ]:
# 2. Stratified K-Fold para Clasificación
print("=== STRATIFIED K-FOLD PARA CLASIFICACIÓN ===")

# Stratified K-Fold mantiene la proporción de clases en cada fold
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

cv_results_stratified = cross_validate(log_reg, X_class_scaled, y_class, 
                                      cv=stratified_kfold, scoring=scoring, 
                                      return_train_score=True)

print("Resultados Stratified K-Fold (5 folds):")
for metric in scoring:
    test_scores = cv_results_stratified[f'test_{metric}']
    train_scores = cv_results_stratified[f'train_{metric}']
    print(f"{metric}:")
    print(f"  Test:  {test_scores.mean():.4f} (+/- {test_scores.std() * 2:.4f})")
    print(f"  Train: {train_scores.mean():.4f} (+/- {train_scores.std() * 2:.4f})")
    print()

# Comparar con K-Fold normal
print("Comparación K-Fold vs Stratified K-Fold:")
print("Accuracy Test Scores:")
print(f"  K-Fold:        {cv_results['test_accuracy'].mean():.4f} (+/- {cv_results['test_accuracy'].std() * 2:.4f})")
print(f"  Stratified:    {cv_results_stratified['test_accuracy'].mean():.4f} (+/- {cv_results_stratified['test_accuracy'].std() * 2:.4f})")


In [ ]:
# 3. Cross Validation para Regresión
print("=== CROSS VALIDATION PARA REGRESIÓN ===")

# Crear el modelo de regresión
lin_reg = LinearRegression()

# K-Fold para regresión
kfold_reg = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

# Métricas para regresión
scoring_reg = ['neg_mean_squared_error', 'neg_mean_absolute_error', 'r2']

cv_results_reg = cross_validate(lin_reg, X_reg_scaled, y_reg, 
                               cv=kfold_reg, scoring=scoring_reg, 
                               return_train_score=True)

print("Resultados K-Fold para Regresión (5 folds):")
for metric in scoring_reg:
    test_scores = cv_results_reg[f'test_{metric}']
    train_scores = cv_results_reg[f'train_{metric}']
    
    # Convertir MSE y MAE a valores positivos para interpretación
    if 'neg_' in metric:
        test_scores = -test_scores
        train_scores = -train_scores
        metric_name = metric.replace('neg_', '')
    else:
        metric_name = metric
    
    print(f"{metric_name}:")
    print(f"  Test:  {test_scores.mean():.4f} (+/- {test_scores.std() * 2:.4f})")
    print(f"  Train: {train_scores.mean():.4f} (+/- {train_scores.std() * 2:.4f})")
    print()


In [ ]:
# 4. Time Series Split (para datos temporales)
print("=== TIME SERIES SPLIT ===")

# Ordenar datos por año para simular serie temporal
df_sorted = df.sort_values('Year').reset_index(drop=True)
X_ts = df_sorted[['Number of Affected Users']].values
y_ts = df_sorted['Financial Loss (in Million $)'].values

# Time Series Split
ts_split = TimeSeriesSplit(n_splits=3)

print("Time Series Split (3 folds):")
for fold, (train_idx, test_idx) in enumerate(ts_split.split(X_ts)):
    X_train_ts, X_test_ts = X_ts[train_idx], X_ts[test_idx]
    y_train_ts, y_test_ts = y_ts[train_idx], y_ts[test_idx]
    
    # Entrenar modelo
    model_ts = LinearRegression()
    model_ts.fit(X_train_ts, y_train_ts)
    
    # Predecir y evaluar
    y_pred_ts = model_ts.predict(X_test_ts)
    mse = mean_squared_error(y_test_ts, y_pred_ts)
    r2 = r2_score(y_test_ts, y_pred_ts)
    
    print(f"Fold {fold + 1}:")
    print(f"  Train size: {len(train_idx)}, Test size: {len(test_idx)}")
    print(f"  MSE: {mse:.4f}, R²: {r2:.4f}")
    print()


In [ ]:
# 5. Grid Search con Cross Validation
print("=== GRID SEARCH CON CROSS VALIDATION ===")

# Definir parámetros para búsqueda
param_grid = {
    'C': [0.1, 1, 10, 100],
    'max_iter': [100, 200, 500]
}

# Crear GridSearchCV
grid_search = GridSearchCV(
    LogisticRegression(random_state=RANDOM_STATE),
    param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    return_train_score=True
)

# Ejecutar búsqueda
grid_search.fit(X_class_scaled, y_class)

print("Mejores parámetros:", grid_search.best_params_)
print("Mejor score:", grid_search.best_score_)
print()

# Mostrar resultados de todos los parámetros
results_df = pd.DataFrame(grid_search.cv_results_)
print("Resultados de Grid Search:")
print(results_df[['param_C', 'param_max_iter', 'mean_test_score', 'std_test_score']].head(10))


In [ ]:
# 6. Validation Curve
print("=== VALIDATION CURVE ===")

# Crear validation curve para el parámetro C de LogisticRegression
param_range = [0.001, 0.01, 0.1, 1, 10, 100, 1000]

train_scores, test_scores = validation_curve(
    LogisticRegression(random_state=RANDOM_STATE),
    X_class_scaled, y_class,
    param_name='C',
    param_range=param_range,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

# Calcular media y desviación estándar
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

# Visualizar validation curve
plt.figure(figsize=(12, 6))
plt.semilogx(param_range, train_mean, 'o-', color='blue', label='Training score')
plt.fill_between(param_range, train_mean - train_std, train_mean + train_std, alpha=0.1, color='blue')
plt.semilogx(param_range, test_mean, 'o-', color='red', label='Cross-validation score')
plt.fill_between(param_range, test_mean - test_std, test_mean + test_std, alpha=0.1, color='red')

plt.xlabel('Parámetro C')
plt.ylabel('Accuracy Score')
plt.title('Validation Curve - Logistic Regression')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Validation Curve completada. El gráfico muestra cómo varía el rendimiento con diferentes valores del parámetro C.")


In [ ]:
# 7. Comparación de Modelos con Cross Validation
print("=== COMPARACIÓN DE MODELOS CON CROSS VALIDATION ===")

# Definir modelos para comparar
models = {
    'Logistic Regression': LogisticRegression(max_iter=200, random_state=RANDOM_STATE),
    'KNN (k=3)': KNeighborsClassifier(n_neighbors=3),
    'KNN (k=5)': KNeighborsClassifier(n_neighbors=5),
    'KNN (k=7)': KNeighborsClassifier(n_neighbors=7)
}

# Evaluar cada modelo
results_comparison = {}

for name, model in models.items():
    scores = cross_val_score(model, X_class_scaled, y_class, cv=5, scoring='accuracy')
    results_comparison[name] = scores
    print(f"{name}: {scores.mean():.4f} (+/- {scores.std() * 2:.4f})")

# Visualizar comparación
plt.figure(figsize=(12, 6))
model_names = list(results_comparison.keys())
model_scores = [results_comparison[name] for name in model_names]

plt.boxplot(model_scores, labels=model_names)
plt.ylabel('Accuracy Score')
plt.title('Comparación de Modelos con Cross Validation')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Encontrar el mejor modelo
best_model = max(results_comparison.items(), key=lambda x: x[1].mean())
print(f"\nMejor modelo: {best_model[0]} con accuracy promedio de {best_model[1].mean():.4f}")


In [ ]:
# 8. Análisis de Varianza en Cross Validation
print("=== ANÁLISIS DE VARIANZA EN CROSS VALIDATION ===")

# Analizar la varianza entre folds para diferentes números de folds
fold_numbers = [3, 5, 10]
variance_analysis = {}

for n_folds in fold_numbers:
    kfold_var = KFold(n_splits=n_folds, shuffle=True, random_state=RANDOM_STATE)
    scores = cross_val_score(LogisticRegression(max_iter=200, random_state=RANDOM_STATE), 
                            X_class_scaled, y_class, cv=kfold_var, scoring='accuracy')
    variance_analysis[n_folds] = scores
    print(f"{n_folds}-Fold CV: Mean={scores.mean():.4f}, Std={scores.std():.4f}")

# Visualizar análisis de varianza
plt.figure(figsize=(12, 6))
fold_labels = [f'{n}-Fold' for n in fold_numbers]
fold_scores = [variance_analysis[n] for n in fold_numbers]

plt.boxplot(fold_scores, labels=fold_labels)
plt.ylabel('Accuracy Score')
plt.title('Análisis de Varianza: Diferentes Números de Folds')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nObservaciones:")
print("- Más folds generalmente reducen la varianza pero aumentan el tiempo de cómputo")
print("- 5-fold es un buen balance entre estabilidad y eficiencia")
print("- La varianza alta puede indicar que el modelo es inestable o que hay datos problemáticos")


In [ ]:
# 9. Resumen y Recomendaciones
print("=== RESUMEN Y RECOMENDACIONES ===")

print("Técnicas de Cross Validation implementadas:")
print("1. K-Fold Cross Validation: División aleatoria en k folds")
print("2. Stratified K-Fold: Mantiene proporción de clases en cada fold")
print("3. Time Series Split: Para datos temporales (no aleatorio)")
print("4. Grid Search: Búsqueda de hiperparámetros con CV")
print("5. Validation Curve: Análisis de sensibilidad de parámetros")
print("6. Comparación de modelos: Evaluación múltiple con CV")
print("7. Análisis de varianza: Estabilidad con diferentes números de folds")

print("\nRecomendaciones:")
print("- Usar Stratified K-Fold para problemas de clasificación desbalanceados")
print("- Time Series Split para datos con dependencia temporal")
print("- 5-fold CV es generalmente suficiente para la mayoría de casos")
print("- Grid Search es útil para optimizar hiperparámetros")
print("- Validation curves ayudan a detectar overfitting/underfitting")
print("- Comparar múltiples modelos para encontrar el mejor")
print("- La varianza alta en CV puede indicar problemas en los datos o modelo")

print("\nMétricas importantes a considerar:")
print("- Accuracy: Proporción de predicciones correctas")
print("- Precision: Proporción de positivos predichos que son realmente positivos")
print("- Recall: Proporción de positivos reales que fueron predichos correctamente")
print("- F1-Score: Media armónica entre precision y recall")
print("- R²: Coeficiente de determinación para regresión")
print("- MSE/MAE: Errores cuadráticos/absolutos medios para regresión")
